# Import libs

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
from models.resnet import ResNet
from models.unet import UNet
from models.skip import skip
from models import get_net
import torch
import torch.optim
# from skimage.measure import compare_psnr
from skimage.metrics import peak_signal_noise_ratio

from skimage.metrics import peak_signal_noise_ratio
from util.common_utils import * 
from util.loss import total_variation

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

PLOT = True
imsize=-1
dim_div_by = 64
dtype = torch.cuda.FloatTensor

# Choose figure

In [ ]:
# import cv2

# video_file = './data/RAW.avi'

# img_np_list = []
# img_gpu_list = []

# cap = cv2.VideoCapture(video_file)

# while (cap.isOpened()):
#     ret, frame = cap.read()
#     if ret == True:
#         img_np = frame[..., 0]
#         img_np = np.expand_dims(img_np, 0)
#         img_np = np.array(img_np, dtype=np.float32) / 255.0
#         img_np_list.append(img_np)
#         img_gpu_list.append(np_to_torch(img_np).type(dtype))
#     else:
#         break

# print("img_np shape: ", img_np_list[0].shape)

# _ = plot_image_grid([img_np_list[0], img_np_list[-1]], factor=4, nrow=2)

In [ ]:
base_dir = './data/building'
# base_dir = './data/power_supply'
img_dir = os.path.join(base_dir, "images_fpn")

file_list = os.listdir(img_dir)
file_list = sorted(file_list, key= lambda x: int(x.split(".")[0]))

img_np_list = []
img_gpu_list = []
img_clean_list = []

for f in file_list:
    _, img_np = get_image(os.path.join(img_dir, f), imsize)
    img_np = img_np[0:1, :, :] 
    img_np_list.append(img_np)
    img_gpu_list.append(np_to_torch(img_np).type(dtype))

print("img_np shape: ", img_np_list[0].shape)

_ = plot_image_grid([img_np_list[0], img_np_list[-1]], factor=4, nrow=2)

In [ ]:
train_frac = 1
test_frac = 1 - train_frac

num_train = int(len(img_np_list) * train_frac)
train_index = np.random.choice(range(len(img_np_list)), num_train, replace=False)
print(train_index)

# Set up everything

In [ ]:
show_every=200
figsize=5
pad = 'reflection' # 'zero'
INPUT = 'noise'
input_depth = 32
OPTIMIZER = 'adam'
OPT_OVER =  'net'

OPTIMIZER = 'adam'


LR = 1e-3
num_iter = 15000
iter_step = 1500

reg_noise_std = 3e-5

NET_TYPE = 'skip'
net = get_net(input_depth, 'skip', pad, n_channels=1,
              skip_n33d=128, 
              skip_n33u=128, 
              skip_n11=4, 
              num_scales=5,
              upsample_mode='bilinear').type(dtype)

# Loss
mse = torch.nn.MSELoss().type(dtype)
#img_var = np_to_torch(img_np).type(dtype)

net_input = get_noise(input_depth, INPUT, img_np.shape[1:]).type(dtype).detach()

# Main loop

In [ ]:
def closure():
    global i, last_net, net_input
    
    if reg_noise_std > 0:
        net_input = net_input_saved + (noise.normal_() * reg_noise_std)
    
    out = net(net_input)

    index = np.random.choice(train_index, size=1)[0]

    target = img_np_list[index]
    target_gpu = img_gpu_list[index]


    if i <= iter_step * 1:
        tv_step = np.random.randint(50, 60)
    elif i <= iter_step * 2:
        tv_step = np.random.randint(40, 50)
    elif i <= iter_step * 3:
        tv_step = np.random.randint(30, 40)
    elif i <= iter_step * 4:
        tv_step = np.random.randint(20, 30)
    elif i <= iter_step * 5:
        tv_step = np.random.randint(10, 20)
    elif i <= iter_step * 6:
        tv_step = np.random.randint(3, 10)
    else:
        tv_step = 1
    
    total_loss = total_variation(img_gpu_list[index] - out, reduction="sum", step=tv_step) #+ 1e-1 * mse(fpn_target, out)
    
    total_loss.backward()

    learned_noise = out.detach().cpu().numpy()[0]
    out_clean = img_np_list[index] - out.detach().cpu().numpy()[0]
    out_clean_norm = (out_clean-np.min(out_clean))/(np.max(out_clean)-np.min(out_clean))
    psrn = peak_signal_noise_ratio(img_np_list[index], out_clean_norm) 

    print ('Iteration %05d    Loss %f PSNR %f' % (i, total_loss.item(), psrn),'\r', end='')
    
    if  PLOT and i % show_every == 0:
        plot_image_grid([np.clip(learned_noise, 0, 1), target, out_clean_norm], factor=figsize, nrow=3)

    i += 1
    return total_loss

# Init globals 
last_net = None
i = 0

net_input_saved = net_input.detach().clone()
noise = net_input.detach().clone()

# Run
p = get_params(OPT_OVER, net, net_input)
optimize(OPTIMIZER, p, closure, LR=LR, num_iter=num_iter)

In [ ]:
# def closure():

#     global i, last_net, net_input, index, num_iter
    
#     if reg_noise_std > 0:
#         net_input = net_input_saved + (noise.normal_() * reg_noise_std)
    
#     out = net(net_input)

#     target = img_np_list[index]
#     target_gpu = img_gpu_list[index]
    
#     #tv_step = np.random.randint(3, 10)
#     tv_step = np.random.randint(5, 30)
#     total_loss = total_variation(img_gpu_list[index] - out, reduction="sum", step=1) + \
#                  total_variation(img_gpu_list[index] - out, reduction="sum", step=tv_step)

#     total_loss.backward()
    
#     learned_noise = out.detach().cpu().numpy()[0]
#     out_clean = img_np_list[index] - out.detach().cpu().numpy()[0]
#     out_clean_norm = (out_clean-np.min(out_clean))/(np.max(out_clean)-np.min(out_clean))
#     psrn = peak_signal_noise_ratio(img_np_list[index], out_clean_norm) 

#     print ('Iteration %05d    Loss %f PSNR %f' % (i, total_loss.item(), psrn),'\r', end='')
    
#     i += 1
#     return total_loss


# out_list = []

# for index, img_np in enumerate(img_np_list):
#     last_net = None
#     i = 0
#     num_iter = 5
#     LR = 1e-4

#     # Run
#     p = get_params(OPT_OVER, net, net_input)
#     optimize(OPTIMIZER, p, closure, LR=LR, num_iter=num_iter)

#     out_np = torch_to_np(net(net_input))
#     out_clean = img_np - out_np[0]
#     out_clean_norm = (out_clean-np.min(out_clean))/(np.max(out_clean)-np.min(out_clean))
#     out_list.append(out_clean_norm)


In [ ]:
out_np = torch_to_np(net(net_input))

out_list = []

for img_np in img_np_list:
    out_clean = img_np - out_np[0]
    out_clean_norm = (out_clean-np.min(out_clean))/(np.max(out_clean)-np.min(out_clean))
    out_list.append(out_clean_norm)

In [ ]:
import cv2
# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = img_np_list[0].shape[2]
frame_height = img_np_list[0].shape[1]
 
print(f"Frame shape: {frame_width} x {frame_height}")
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter('out.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 5, (frame_width, frame_height), False)
 
for index, out_img in enumerate(out_list):
    # Write the frame into the file 'output.avi'
    frame = out_img[0, ...] 
    frame = frame * 255
    frame = np.array(frame, dtype=np.uint8)

    # Display the resulting frame    
    cv2.imshow('frame',frame)
 
    # Press Q on keyboard to stop recording
    if cv2.waitKey(100) & 0xFF == ord('q'):
      break
    out.write(frame)
    
    cv2.imwrite("out-tv-dip-{}".format(file_list[index]), frame)

out.release()
# Closes all the frames
cv2.destroyAllWindows()